In [29]:
import numpy as np
import heapq 
from collections import namedtuple
import random
from random import choice
from tqdm import tqdm

np.random.seed(1)
random.seed(1)

action = namedtuple('Action', ['pos1', 'pos2'])
 
# Define puzzle dimension
PUZZLE_DIM = 4
RANDOMIZE_STEPS = 100_000

In [30]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action) -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state



def manhattan_distance(state: np.ndarray, goal: np.ndarray) -> int: 
    distance = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:   
                goal_x, goal_y = divmod(value - 1, PUZZLE_DIM)  
                distance += abs(x - goal_x) + abs(y - goal_y)   
    return distance

In [31]:
 

def a_star(start_state: np.ndarray, target_state: np.ndarray):
    open_list = []
    explored_states = set()
    parent_map = {}
    state_cost = {}
    
    # Convert states to tuples
    start_state_tuple = tuple(start_state.flatten())
    target_state_tuple = tuple(target_state.flatten())
    
    # Heuristic cost (Manhattan distance)
    initial_heuristic = manhattan_distance(start_state, target_state)
    state_cost[start_state_tuple] = 0
    
    # Add the start state to the open list
    heapq.heappush(open_list, (initial_heuristic, start_state_tuple))
    explored_states.add(start_state_tuple)
    
    iteration_count = 0  
    
    while open_list:
        iteration_count += 1

        # Get the state with the lowest f value
        _, current_state = heapq.heappop(open_list)
        
        # Goal check
        if current_state == target_state_tuple:
            solution_path = []
            state = current_state
            while state != start_state_tuple:
                solution_path.append(state)
                state = parent_map[state]
            solution_path.reverse()
            return solution_path, state_cost[current_state], iteration_count
        
        explored_states.add(current_state)
        
        reshaped_state = np.array(current_state).reshape((PUZZLE_DIM, PUZZLE_DIM))
        for move in available_actions(reshaped_state):
            next_state = do_action(reshaped_state, move)
            next_state_tuple = tuple(next_state.flatten())
            
            if not next_state_tuple in explored_states:
                new_g_value = state_cost[current_state] + 1
                new_h_value = manhattan_distance(next_state, target_state)
                new_f_value = new_g_value + new_h_value
                
                # Update if it's a better path
                if next_state_tuple not in state_cost or new_g_value < state_cost[next_state_tuple]:
                    state_cost[next_state_tuple] = new_g_value
                    heapq.heappush(open_list, (new_f_value, next_state_tuple))
                    parent_map[next_state_tuple] = current_state
    
    print("No found")
    return None



In [32]:
# Randomize initial state
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
initial_state = goal_state.copy()
for _ in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    initial_state = do_action(initial_state, choice(available_actions(initial_state)))
 
# Solve the puzzle
print("Initial state:")
print(initial_state)
solution_path, steps, iterations = a_star(initial_state, goal_state)


print(f"number of iterations: {iterations}")
print(f"number of steps to solve puzzle: {steps}")


Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 306869.46it/s]


Initial state:
[[10  8  5  9]
 [11  7  3  0]
 [13  2 12 15]
 [ 6 14  1  4]]
number of iterations: 7128359
number of steps to solve puzzle: 54
